In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("covid_toy.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [5]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['has_covid']), df['has_covid'], test_size = 0.2, random_state = 0)

X_train.head()

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore


In [15]:
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

In [16]:
X_train_fever.shape

(80, 1)

In [17]:
ohe = OneHotEncoder(drop = 'first', sparse = False, dtype = np.int32)
X_train_ohe = ohe.fit_transform(X_train[['gender','city']])
X_test_ohe = ohe.fit_transform(X_test[['gender', 'city']])

C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [18]:
oe = OrdinalEncoder(categories = [['Mild','Strong']])

X_train_oe = oe.fit_transform(X_train[['cough']])
X_test_oe = oe.fit_transform(X_test[['cough']])

In [19]:
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

In [21]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_ohe, X_train_oe), axis = 1)

X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_ohe, X_test_oe), axis = 1)

In [22]:
X_train_transformed.shape

(80, 7)

In [23]:
from sklearn.compose import ColumnTransformer

In [25]:
transformer = ColumnTransformer(transformers = [
    ('trf1', SimpleImputer(),['fever']),
    ('trf2', OrdinalEncoder(categories = [['Mild','Strong']]),['cough']),
    ('trf3', OneHotEncoder(sparse = False, drop = 'first', dtype = np.int32),['gender','city'])
], remainder = 'passthrough')

In [28]:
transformer.fit_transform(X_train).shape

C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [29]:
transformer.fit_transform(X_test).shape

C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(20, 7)